In [1]:
%run "./utils.ipynb"

Using TensorFlow backend.


# Main configs

In [2]:
loadFromMerged=True
loadFromIndexes= False
Mapper='S'
IgnoreEmpty= True
FoldID ="1"
Epoch_count=100
Batch_size=5000
INCLUDE_DIRECTION = True

LOAD_OLD_DATA_TRAIN = False
LOAD_OLD_DATA_TEST = False
MERGE_TESTS=True

TEST_CLASS_CAP =999999

pre_Process_remove_empties = True


# per Karthika's change
SKIP_LOCK_FILE = True

# 
LOAD_HOMES_ONLY = True

test_file_names = ["home_sk_final.json"] 
train_file_names = ["home_os_final.json"]

train_x_path = '../files/iot_data_2020/train/pcap_segments/'
train_y_path = '../files/iot_data_2020/train/hub_segments/'

test_y_path = '../files/iot_data_2020/usecases/hub_segments_final/'
test_x_path = '../files/iot_data_2020/usecases/pcap_segments_final/'

In [3]:
x= []
y= []

x_test = []
y_test = []
y_test_service= []

x_train = []
y_train = []
y_train_service = []

test_names = []

add_to_trainig = [0,2]

if loadFromMerged:
    print("in load from merged")
    if LOAD_OLD_DATA_TRAIN:
        with open(  '../files/train/test/test_homes/final_upload/Merged_final_with_home/merged_hub_segments_final.json'  ) as f:
            y_data = json.load(f)

        with open(  '../files/train/test/test_homes/final_upload/Merged_final_with_home/merged_pcap_segments_final.json'  ) as f:
            x_data = json.load(f)

    #     with open(  '../files/train/merged/hub_segments_2.json'  ) as f:
    #         y_data = json.load(f)

    #     with open(  '../files/train/merged/pcap_segments_2.json'  ) as f:
    #         x_data = json.load(f)

        if len( y_data ) != len(x_data) :
            print( pick )


        x_train,y_train, y_train_service= clean_data( x_data, y_data, IgnoreEmpty , 
                                                     Mapping=Mapper, include_direction=INCLUDE_DIRECTION )
    else:
        the_dir = test_x_path if LOAD_HOMES_ONLY else train_x_path
        the_dir = os.path.join(the_dir, '*.json')
        for pick in sorted(glob.glob( the_dir )):
            fname  = os.path.basename(pick)
            if len(train_file_names) > 0 and fname not in train_file_names:
                print("ignoring file : %s" % fname)
                continue
#             test_names.append( fname )
            with open( os.path.join(test_y_path if LOAD_HOMES_ONLY else train_y_path, fname) ) as f:
                y_data = json.load(f)

            with open( os.path.join(test_x_path if LOAD_HOMES_ONLY else train_x_path, fname) ) as f:
                x_data = json.load(f)

            if len( y_data ) != len(x_data) :
                print( pick )
                continue

            x_t,y_t, y_t_s= clean_data( x_data, y_data, IgnoreEmpty , 
                                                     Mapping=Mapper, include_direction=INCLUDE_DIRECTION )
    
            x_train.extend(x_t)
            y_train.extend(y_t)
            y_train_service.extend(y_t_s)
            

    #     continue
#     if loadFromIndexes:
#         print("load from indexes")
#         with open("../files/train/merged/items_2_test-train_indexes.json")  as f:
#             index_info = json.load(f)


#         for i in index_info[FoldID]["test"]:
#             x_test[str(i)]=(x_data[str(i)] )
#             y_test[str(i)]=(y_data[str(i)] )

#         for i in index_info[FoldID]["train"]:
#             x_train[str(i)]=(  x_data[str(i)] )
#             y_train[str(i)]=(  y_data[str(i)] )
        
#         x_test_t,y_test_t= clean_data( x_test, y_test, IgnoreEmpty , Mapping=Mapper)
#         x_test.append(x_test_t)
#         y_test.append(y_test_t)
    #     else :
    print("loading from test files")
    files_path = '../files/train/test/test_homes/final_upload/usecases/pcap_segments_final_final/home*.json' if LOAD_OLD_DATA_TEST else '../files/iot_data_2020/usecases/pcap_segments_final/*.json'
    test_y_dir = '../files/train/test/test_homes/final_upload/usecases/hub_segments_final_final/'  if LOAD_OLD_DATA_TEST else '../files/iot_data_2020/usecases/hub_segments_final/'
    test_x_dir = '../files/train/test/test_homes/final_upload/usecases/pcap_segments_final_final/' if LOAD_OLD_DATA_TEST else '../files/iot_data_2020/usecases/pcap_segments_final/'
    
    test_files = [ x for x in sorted(glob.glob(files_path)) if not SKIP_LOCK_FILE or ( SKIP_LOCK_FILE and  'lock' not in x )]
    print( "found files : " , len(test_files) )
    
    
    for pick  in test_files:
        fname  = os.path.basename(pick)
        if len(test_file_names) > 0 and fname not in test_file_names:
            print("ignoring file : %s" % fname)
            continue
        test_names.append( fname )
        print(pick)
        with open( os.path.join(test_y_dir , fname) ) as f:
            y_data_test = json.load(f)

        with open( os.path.join(test_x_dir, fname) ) as f:
            x_data_test = json.load(f)


        t_x,t_y, t_z= clean_data( x_data_test, y_data_test, False , Mapping=Mapper, include_direction= INCLUDE_DIRECTION )

#         if test_files.index(pick) in add_to_trainig:
#             x_test_t,y_test_t, y_test_service_t= clean_data( x_data_test, y_data_test, IgnoreEmpty , Mapping=Mapper)
#             x_train.extend(x_test_t)
#             y_train.extend(y_test_t)
#             y_train_service.extend(y_test_service_t)

                
        if MERGE_TESTS:
            x_test.extend(t_x)
            y_test.extend(t_y)
            y_test_service.extend(t_z)
        else:
            x_test.append(t_x)
            y_test.append(t_y)
            y_test_service.append(t_z)
if MERGE_TESTS:
    x_test = [x_test]
    y_test = [y_test]
    y_test_service = [y_test_service]
    test_names =['MERGED']
#     x_test = x_data[ index_info["1"]["test"]  ]
#     y_test = y_data[ index_info["1"]["test"]  ]
    
#     x_train = x_data[ index_info["1"]["train"]  ]
#     y_train = y_data[ index_info["1"]["train"]  ]
#     x.extend(t_x)
#     y.extend(t_y)
# else:
#     for pick in sorted(glob.glob( '../files/train/hub_segments/*.json' )):
#         fname  = os.path.basename(pick)
#         test_names.append( fname )
#         with open( os.path.join( '../files/train/hub_segments/', fname) ) as f:
#             y_data = json.load(f)

#         with open( os.path.join('../files/train/pcap_segments/', fname) ) as f:
#             x_data = json.load(f)

#         if len( y_data ) != len(x_data) :
#             print( pick )
#             continue

#         t_x,t_y= clean_data( x_data, y_data, True, include_direction=INCLUDE_DIRECTION )

#         x.extend( t_x)
#         y.extend(t_y)

x= np.array(x)
y= np.array(y)

# x_train = np.append( x_train, x_test[0] , axis=0)
# x_train = np.append( x_train, x_test[2] , axis=0)

# y_train = np.append( y_train, y_test[0] , axis=0)
# y_train = np.append( y_train, y_test[2] , axis=0)


len(x_train), len(y_test)

in load from merged
18308 13614
ignoring file : home_sk_final.json
ignoring file : test_data_light.json
ignoring file : test_data_lock.json
ignoring file : test_data_motion_2.json
loading from test files
found files :  4
ignoring file : home_os_final.json
../files/iot_data_2020/usecases/pcap_segments_final/home_sk_final.json
5837 5837
ignoring file : test_data_light.json
ignoring file : test_data_motion_2.json


(13614, 1)

# sets the classes 

In [4]:
classes = sorted(list(np.unique(  np.concatenate( y_train  ))))
# print([ (i , classes[i]) for i in range( len(classes) ) ])

service_classes = sorted(list(np.unique(  np.concatenate( y_train_service  ))))
[ (i , service_classes[i]) for i in range( len(service_classes) ) ]

[(0, 'acceleration'),
 (1, 'activity'),
 (2, 'battery'),
 (3, 'button'),
 (4, 'colorTemperature'),
 (5, 'contact'),
 (6, 'level'),
 (7, 'lock'),
 (8, 'motion'),
 (9, 'ping'),
 (10, 'status'),
 (11, 'switch'),
 (12, 'temperature'),
 (13, 'threeAxis'),
 (14, 'unknown')]

In [5]:
classes

['acceleration',
 'activity',
 'battery',
 'button',
 'colorTemperature',
 'contact',
 'level',
 'lock',
 'motion',
 'ping',
 'status',
 'switch',
 'temperature',
 'threeAxis',
 'unknown']

## Filter the records by service/event types

choose services to keep : 

In [6]:
if Mapper=='SE':
    services_to_keep = ['button-held',
 'button-pushed',
        "colorTemperature-XXX",
"contact-closed",
"contact-open",
"level-XXX",
"lock-locked",
"lock-unlocked",
"motion-active",
"motion-inactive",
"ping-ping",
"status-closed",
"status-open",
"switch-off",
"switch-on",
"temperature-XXX"
    ] 
else:
     services_to_keep =["button",
"colorTemperature",
"contact",
"level",
"lock",
"motion",
"ping",
"status",
"switch",
"temperature"]

# keep all ? 
# services_to_keep= classes

find records which need change

In [7]:
train_indexes_to_change =  [ i for i in range(len(y_train)) if
              is_clean( y_train[i],to_keep=services_to_keep, return_clean=False) ] 

y_train = [ ['unknown'] if i in train_indexes_to_change else y_train[i] 
           for i in range(len(y_train)) ] 

for t_index in range(len(y_test)):
    test_indexes_to_change =  [ i for i in range(len(y_test[t_index])) if
                  is_clean( y_test[t_index][i],to_keep=services_to_keep, return_clean=False) ] 

    y_test[t_index] = [ ['unknown'] if i in test_indexes_to_change else y_test[t_index][i] 
               for i in range(len(y_test[t_index])) ] 

In [8]:
def make_known_unknown(inp, unknown_ind):
    return [ [1,0] if (x[unknown_ind] == 1 ) else [0,1]   for x in inp ]

known_unknown_y_train = [ [1,0] if (len(x) == 1 and ("unknown" in x or 'unknown-' in x)) else [0,1]   for x in y_train ]

known_unknown_y_test= [] 

for i in range(len(y_test)):
    
    known_unknown_y_test.append( [ [1,0] if (len(x) == 1 and "unknown" in x) else [0,1]   for x in y_test[i] ])


remove classes that are ignored from the services list 

In [9]:
classes = services_to_keep
classes.append('unknown')
classes

['button',
 'colorTemperature',
 'contact',
 'level',
 'lock',
 'motion',
 'ping',
 'status',
 'switch',
 'temperature',
 'unknown']

In [10]:
# x,y,_=pre_process_raw( x_train, y_train , 15, zero_pad=True, normalize=True, classes=classes, remove_empty=pre_Process_remove_empties)

In [11]:
_,_,temp = pre_process_raw( x_test[0], y_test[0] , 13, zero_pad=False, normalize=False, classes=classes, as_string=True, class_cap=TEST_CLASS_CAP,remove_empty=True)
indexes_to_keep = temp[1]


get the data and preprocess train and test portions

In [12]:
dim_size= 20
x_random_forest_train,y_random_forest_train, _ = pre_process_raw( x_train, y_train , dim_size, zero_pad=False, normalize=False, classes=classes, as_string=True,remove_empty=pre_Process_remove_empties)
rf_test = [] 

rf_tests  = [ pre_process_raw( x_test[i], y_test[i] , dim_size, zero_pad=False, normalize=False, classes=classes, as_string=True, class_cap=TEST_CLASS_CAP,remove_empty=pre_Process_remove_empties) for i in range(len(x_test)) ] 

In [13]:
for i in range(len(rf_tests)):
    print(len(rf_tests[i][1]), len(y_test[i]))

5837 5837


In [14]:
y_test[0][3049][0]==classes[-2]

False

In [15]:
 y_test[0][3049], classes, rf_tests[0][1][3049]

(array(['ping', 'unknown'], dtype='<U7'),
 ['button',
  'colorTemperature',
  'contact',
  'level',
  'lock',
  'motion',
  'ping',
  'status',
  'switch',
  'temperature',
  'unknown'],
 array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]))

In [16]:
len(rf_tests[0][0]), len(x_test[0]), len(y_test[0])

(5837, 5837, 5837)

vectorize the X vars

In [17]:
vectorizer = CountVectorizer(analyzer='word', ngram_range=(1, 5))
x_xgb_train = vectorizer.fit_transform(x_random_forest_train)
xgb_test = []
for x in range(len(rf_tests)):
    xgb_test.append( ( vectorizer.transform(rf_tests[x][0]),
                    rf_tests[x][1],
                    rf_tests[x][2]
                  ))
    

# XGBoost 

In [18]:

xgb_known_unknown_separator_classifier = OneVsRestClassifier(XGBClassifier())

xgb_known_unknown_separator_classifier.fit(x_xgb_train, np.array(known_unknown_y_train))

train_known_unknown_pred=xgb_known_unknown_separator_classifier.predict_proba(x_xgb_train)


In [19]:
test_known_unknown_predicted = []
for i in range(len(xgb_test)) :
    print( "==================HOME Case : %s =============" % test_names[ i] )
    xgb_pred=xgb_known_unknown_separator_classifier.predict( xgb_test[i][0])
    test_known_unknown_predicted.append(xgb_pred)
    print_info( np.array( make_known_unknown(xgb_test[i][1], classes.index('unknown'))), xgb_pred, ["unknown","known"])

==================HOME Case : MERGED =============
                         Class  Accuracy     Recall  Precision   F Score    Count            TP/TN/FP/FN
------------------------------------------------------------------------
                       unknown     0.926      0.938     0.920     0.929      2997  2811/ 2594/  246/  186
                         known     0.926      0.913     0.933     0.923      2840  2594/ 2811/  186/  246
------------------------------------------------------------------------
                      AVERAGES     0.926      0.926     0.926     0.926      5837     0/    0/    0/    0
------------------------------------------------------------------------
             Weighted AVERAGES     0.926      0.926     0.926     0.926      5837     0/    0/    0/    0
------------------------------------------------------------------------
       known Weighted AVERAGES     0.926      0.913     0.933     0.923      5837     0/    0/    0/    0
----------------------

In [20]:
len(test_known_unknown_predicted[0]), len(xgb_test[0]), (xgb_test[0][1]).shape

(5837, 3, (5837, 11))

In [21]:
known_indexes_train  = [ i for i in range(len(train_known_unknown_pred)) if train_known_unknown_pred[i][1] ==1 ]

x_train_known = x_xgb_train[known_indexes_train]
y_train_known = y_random_forest_train[known_indexes_train]

xgb_test_known = [] 

for test_index in range(len(test_known_unknown_predicted)):
    known_indexes= [ i for i in range(len(test_known_unknown_predicted[test_index])) if test_known_unknown_predicted[test_index][i][1] ==1 ]
    
    xgb_test_known.append(  (xgb_test[test_index][0][known_indexes], 
                            xgb_test[test_index][1][known_indexes],
                            xgb_test[test_index][2],
                            known_indexes
                           ))

In [22]:
xgb_classifier = OneVsRestClassifier(XGBClassifier())

xgb_classifier.fit(x_xgb_train, np.array(y_random_forest_train))


OneVsRestClassifier(estimator=XGBClassifier(base_score=None, booster=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, gamma=None,
                                            gpu_id=None, importance_type='gain',
                                            interaction_constraints=None,
                                            learning_rate=None,
                                            max_delta_step=None, max_depth=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=100, n_jobs=None,
                                            num_parallel_tree=None,
                                            objective='binary:logistic',
                                            

In [23]:
xg_boost_results = []
xgb_preds = []
for i in range(len(xgb_test_known)) :
    print( "==================HOME Case : %s =============" % test_names[ i] )
    xgb_pred= xgb_classifier.predict( xgb_test_known[i][0])
    xgb_pred = add_unknowns_back(xgb_pred,xgb_test[i][1], xgb_test_known[i][3] , classes)
    xgb_preds.append(xgb_pred)
    xg_boost_results.append(print_info( xgb_test[i][1], xgb_pred, classes, confidance=0.8))

==================HOME Case : MERGED =============


/opt/anaconda3/envs/omid/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':
/opt/anaconda3/envs/omid/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
/opt/anaconda3/envs/omid/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in double_scalars
  del sys.path[0]
/opt/anaconda3/envs/omid/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/omid/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels

                         Class  Accuracy     Recall  Precision   F Score    Count            TP/TN/FP/FN
------------------------------------------------------------------------
                        button     0.993      0.204     1.000     0.339        49    10/ 5788/    0/   39
              colorTemperature     1.000      1.000     1.000     1.000         1     1/ 5836/    0/    0
                       contact     0.989      0.658     0.906     0.763       161   106/ 5665/   11/   55
                         level     0.993      0.340     0.783     0.474        53    18/ 5779/    5/   35
                          lock     1.000        nan       nan       nan         0     0/ 5837/    0/    0
                        motion     0.960      0.511     0.289     0.370       133    68/ 5537/  167/   65
                          ping     0.995      0.995     0.993     0.994      2301  2290/ 3520/   16/   11
                        status     0.997      0.912     0.912     0.912       10

In [24]:
# for i in range(len(rf_test_known)) :
#     print( "==================HOME Case : %s =============" % test_names[ i] )
#     rf_pred= xgb_classifier.predict( rf_test_known[i][0])
#     print_info( rf_test_known[i][1], rf_pred, classes, confidance=0.3)

# Random Forest 

## string encoding test

In [25]:
from sklearn.ensemble import RandomForestClassifier

knownity_rf =  RandomForestClassifier(n_estimators=100, max_depth=400,
                             random_state=0 )
knownity_rf_results = knownity_rf.fit(x_xgb_train, known_unknown_y_train)


In [26]:
train_known_unknown_pred=knownity_rf.predict(x_xgb_train)

In [27]:
test_known_unknown_predicted = []
for i in range(len(rf_tests)) :
    print( "==================HOME Case : %s =============" % test_names[ i] )
    rf_pred=knownity_rf.predict( xgb_test[i][0])
    test_known_unknown_predicted.append(rf_pred)
    print_info( np.array( make_known_unknown(rf_tests[i][1], classes.index('unknown'))), rf_pred, ["unknown","known"])

==================HOME Case : MERGED =============
                         Class  Accuracy     Recall  Precision   F Score    Count            TP/TN/FP/FN
------------------------------------------------------------------------
                       unknown     0.927      0.939     0.919     0.929      2997  2815/ 2593/  247/  182
                         known     0.927      0.913     0.934     0.924      2840  2593/ 2815/  182/  247
------------------------------------------------------------------------
                      AVERAGES     0.927      0.926     0.927     0.926      5837     0/    0/    0/    0
------------------------------------------------------------------------
             Weighted AVERAGES     0.927      0.927     0.927     0.926      5837     0/    0/    0/    0
------------------------------------------------------------------------
       known Weighted AVERAGES     0.927      0.913     0.934     0.924      5837     0/    0/    0/    0
----------------------

In [28]:
type(rf_tests[0][1])

numpy.ndarray

In [29]:
known_indexes_train  = [ i for i in range(len(train_known_unknown_pred)) if train_known_unknown_pred[i][1] ==1 ]

x_train_known = x_xgb_train[known_indexes_train]
y_train_known = y_random_forest_train[known_indexes_train]

rf_test_known = [] 

for test_index in range(len(test_known_unknown_predicted)):
    known_indexes= [ i for i in range(len(test_known_unknown_predicted[test_index])) if test_known_unknown_predicted[test_index][i][1] ==1 ]
    
    rf_test_known.append(  (xgb_test[test_index][0][known_indexes], 
                            rf_tests[test_index][1][known_indexes],
                            rf_tests[test_index][2],
                            known_indexes
                           ))

In [30]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=200, max_depth=400,
                             random_state=0 )
t_hist = clf.fit(x_train_known, y_train_known)


In [31]:
rf_results = []
rf_preds = [] 
for i in range(len(rf_test_known)) :
    print( "==================HOME Case : %s =============" % test_names[ i] )
    rf_pred= clf.predict( rf_test_known[i][0])
    rf_pred = add_unknowns_back(rf_pred,rf_tests[i][1], rf_test_known[i][3] , classes)
    rf_preds.append(rf_pred)
    rf_results.append(print_info( rf_tests[i][1], rf_pred, classes, confidance=0.09))

==================HOME Case : MERGED =============


/opt/anaconda3/envs/omid/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
/opt/anaconda3/envs/omid/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':
/opt/anaconda3/envs/omid/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in double_scalars
  del sys.path[0]
/opt/anaconda3/envs/omid/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/omid/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels

                         Class  Accuracy     Recall  Precision   F Score    Count            TP/TN/FP/FN
------------------------------------------------------------------------
                        button     0.993      0.184     1.000     0.310        49     9/ 5788/    0/   40
              colorTemperature     1.000      0.000       nan     0.000         1     0/ 5836/    0/    1
                       contact     0.989      0.640     0.954     0.766       161   103/ 5671/    5/   58
                         level     0.991      0.057     0.750     0.105        53     3/ 5783/    1/   50
                          lock     1.000        nan       nan       nan         0     0/ 5837/    0/    0
                        motion     0.961      0.466     0.281     0.350       133    62/ 5545/  159/   71
                          ping     0.996      0.997     0.992     0.995      2301  2294/ 3518/   18/    7
                        status     0.998      0.892     0.978     0.933       10

In [ ]:
# for i in range(len(rf_test_known)) :
#     print( "==================HOME Case : %s =============" % test_names[ i] )
#     rf_pred= clf.predict( rf_test_known[i][0])
#     print_info( rf_test_known[i][1], rf_pred, classes, confidance=0.8)

## Random Forest baseline calc

preproicess the data 

In [ ]:
dim_size= 50
x_random_forest_train,y_random_forest_train, _ = pre_process_raw( x_train, y_train , dim_size, zero_pad=True, normalize=False, classes=classes,remove_empty=pre_Process_remove_empties)
rf_tests  = [ pre_process_raw( x_test[i], y_test[i] , dim_size, zero_pad=True, normalize=False, classes=classes, class_cap=TEST_CLASS_CAP,remove_empty=pre_Process_remove_empties) for i in range(len(x_test)) ] 

In [ ]:
len(rf_tests[0][0]),len( rf_tests[0][1]), len(x_test[0]), len(y_test[0])

first stage RF will learn if it is a known or unknown instance 

In [ ]:
from sklearn.ensemble import RandomForestClassifier

knownity_rf =  RandomForestClassifier(n_estimators=960, max_depth=9050,
                             random_state=0 )
knownity_rf_results = knownity_rf.fit(x_random_forest_train, known_unknown_y_train)


In [ ]:
train_known_unknown_pred=knownity_rf.predict(x_random_forest_train)

In [ ]:
test_known_unknown_predicted = []
for i in range(len(rf_tests)) :
    print( "==================HOME Case : %s =============" % test_names[ i] )
    rf_pred=knownity_rf.predict( rf_tests[i][0])
    test_known_unknown_predicted.append(rf_pred)
    print_info( np.array( make_known_unknown(rf_tests[i][1], classes.index('unknown'))), rf_pred, ["unknown","known"])

In [ ]:
known_indexes_train  = [ i for i in range(len(train_known_unknown_pred)) if train_known_unknown_pred[i][1] ==1 ]

x_train_known = x_random_forest_train[known_indexes_train]
y_train_known = y_random_forest_train[known_indexes_train]

rf_test_known = [] 

for test_index in range(len(rf_tests)):
    known_indexes= [ i for i in range(len(test_known_unknown_predicted[test_index])) if test_known_unknown_predicted[test_index][i][1] ==1 ]
    print(test_index, max(known_indexes), len(rf_tests[test_index][0]),len(known_indexes))
    rf_test_known.append(  (rf_tests[test_index][0][known_indexes], 
                            rf_tests[test_index][1][known_indexes],
                            rf_tests[test_index][1], 
                           known_indexes))

# CNN

In [ ]:
# create the model
dim_size =20
x_lstm_prossed_train,y_lstm_prossed_train, _ = pre_process_raw( x_train, y_train , dim_size, zero_pad=False, normalize=False,classes=classes, as_string=True,remove_empty=pre_Process_remove_empties)
_, y_s_lstm_processed_train ,_ =  pre_process_raw( x_train, y_train_service , dim_size, zero_pad=False, normalize=False,classes=service_classes, as_string=True,remove_empty=pre_Process_remove_empties)
# x_lstm_prossed_test,y_lstm_prossed_test, _ = pre_process_raw( x_test, y_test_2 , dim_size, zero_pad=True, normalize=False,classes=classes)
lstm_tests  = [ pre_process_raw( x_test[i], y_test[i] , dim_size, zero_pad=False, normalize=False, classes=classes, class_cap=TEST_CLASS_CAP,remove_empty=pre_Process_remove_empties) for i in range(len(x_test)) ] 
lstm_tests_services  = [ pre_process_raw( x_test[i], y_test_service[i] , dim_size, zero_pad=False, normalize=True, classes=service_classes, class_cap=TEST_CLASS_CAP,remove_empty=pre_Process_remove_empties) for i in range(len(x_test)) ] 



In [ ]:
# known_indexes_train  = [ i for i in range(len(train_known_unknown_pred)) if train_known_unknown_pred[i][1] ==1 ]

# # x_lstm_prossed_train = x_lstm_prossed_train[known_indexes_train]
# y_lstm_prossed_train = y_lstm_prossed_train[known_indexes_train]

# lstm_tests_known = [] 

# for test_index in range(len(rf_tests)):
#     known_indexes= [ i for i in range(len(test_known_unknown_predicted[test_index])) if test_known_unknown_predicted[test_index][i][1] ==1 ]
    
#     lstm_tests_known.append(  (lstm_tests[test_index][0][known_indexes], 
#                             lstm_tests[test_index][1][known_indexes],
#                             lstm_tests[test_index][2]))

In [ ]:
# x_lstm_prossed_test2 = np.expand_dims(x_lstm_prossed_test,axis=1)
# x_lstm_prossed_train2 =np.expand_dims(x_random_forest_train,axis=1)
# x_lstm_prossed_train2 =x_random_forest_train


# x_lstm_prossed_test2 = x_lstm_prossed_test.reshape(len(x_lstm_prossed_test),dim_size,1)
x_lstm_prossed_train2 =x_train_known.reshape((x_train_known.shape[0],x_train_known.shape[1],1))

# y_lstm_prossed_test2 = y_lstm_prossed_test.reshape(len(y_lstm_prossed_test),len(classes),1)
# y_lstm_prossed_train2 =y_lstm_prossed_train.reshape(len(y_lstm_prossed_train),len(classes),1)

In [ ]:
lstm_tests_known = []
for tt  in range( len(rf_test_known) ):
    lstm_tests_known.append( (rf_test_known[tt][0].reshape(rf_test_known[tt][0].shape[0],
                                                     rf_test_known[tt][0].shape[1],
                                                     1) ,
                           rf_test_known[tt][1],
                           rf_test_known[tt][2] ))

In [ ]:
# classes.index('ping'), classes.index('unknown'), len(classes), len(y_lstm_prossed_train[0])
# ping=classes.index('ping')
# unknown =classes.index('unknown')
# key = np.ones_like(y_lstm_prossed_train[0])
# key[unknown]=0
# key[ping]=0
# key

In [ ]:
len(x_lstm_prossed_train2),len( y_lstm_prossed_train)

In [ ]:
inputs  = Input(  (x_lstm_prossed_train2[0].shape[0]  ,1) )

out = Conv1D(128,3,padding='same')(inputs)
out = BatchNormalization()(out)
out = Activation('relu')(out)
out = Dropout(0.2)(out)
out = Conv1D(64,3,padding='same')(out)

dout_1  = Dropout(0.2)(out)
# flt_1   = Flatten()(dout_1)
dense_1 = Dense(32, activation='relu')(dout_1)
# dout_3  = Dropout(0.2)(dense_1)
flt_1   = Flatten()(dense_1)

out_put_final = Dense(len(classes), activation='sigmoid', name='Event_output')(flt_1)


model2 = Model(inputs=[inputs], outputs=[out_put_final])


losses = {
    "Event_output": f1_loss_perClass ,
#     "Event_output": f1_loss_perRow 
    "Event_output": "categorical_crossentropy",
}
lossWeights = {"Event_output": 200,
#                "Event_output": 30.0 
    "Event_output": 5
              }
 


model2.compile(loss=losses,loss_weights=lossWeights, optimizer='adam', metrics=[f1_perRow,f1_perClass,'acc'])
# model2.compile(loss=losses, loss_weights=lossWeights, optimizer='adam', metrics=['accuracy'])

checkpoint = ModelCheckpoint('IoTDownNet_cnn_newloss', monitor='loss', verbose=0, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

print(model2.summary())
# hist2 = model2.fit(x_lstm_prossed_train2, y_train_known, epochs=70, batch_size=2000, shuffle=True)

In [ ]:

losses = {
    "Event_output": f1_loss_perClass ,
    "Event_output": f1_loss_perRow ,
    "Event_output": "categorical_crossentropy",
}
lossWeights = {
    "Event_output": 300,
               "Event_output": 300.0 ,
    "Event_output": 20
              }
 


model2.compile(loss=losses,loss_weights=lossWeights, optimizer='adam', metrics=[f1_perRow,f1_perClass,'acc'])


In [ ]:
# hist2 = model2.fit(x_lstm_prossed_train2, y_train_known, epochs=100, batch_size=7010, shuffle=True, callbacks=callbacks_list)

In [ ]:
# hist2 = model2.fit(x_lstm_prossed_train2, y_train_known, epochs=200, batch_size=7000, shuffle=True, callbacks=callbacks_list)

In [ ]:
# hist2 = model2.fit(x_lstm_prossed_train2, y_train_known, epochs=300, batch_size=7000, shuffle=True, callbacks=callbacks_list)

In [ ]:
# model2.load_weights('IoTDownNet_cnn_nocca')

In [ ]:
# lstm_results = []
# lstm_preds = [] 
# for i in range(len(lstm_tests_known)) :
#     print( "==================HOME Case : %s =============" % test_names[ i] )
#     lstm_pred= model2.predict( lstm_tests_known[i][0])
#     lstm_pred = add_unknowns_back(lstm_pred,rf_tests[i][1], rf_test_known[i][3] , classes)
#     lstm_preds.append(lstm_pred)
#     lstm_results.append(print_info( rf_tests[i][1], lstm_pred, classes, confidance=0.09))

## load Seq2Seq results

In [ ]:
def label_2_vec(y_data, classes):
    ret = []
    
    for i,x in enumerate(classes):
        if 'unknown' in x:
            unknown_index = i
            break
    
    for i,x in enumerate(y_data):
        temp = np.zeros(len(classes))
        x= list(set(x.split(' ')))
        for y in x: 
            if y in classes:
                temp[classes.index(y)] = 1
                
        if np.sum(temp) == 0 :
            temp[unknown_index] = 1
        ret.append(temp)
    return ret
            

load the test fileds from seq2seq output

In [ ]:
seq2seq_results = []
for test_case in test_files:
    file_name = os.path.join("../files/seq2seq/%s_test_services_to_keep_%d_pred.json" % (os.path.basename(test_case)[:-5], 0 if Mapper=='S' else 2))
    with open(file_name, 'r') as file:
        results = file.readlines()
        results = [x.strip() for x in results]
        results = label_2_vec(results, classes)
        if MERGE_TESTS:
            seq2seq_results.extend(results)
        else:
            seq2seq_results.append(results)

if MERGE_TESTS:
    seq2seq_results = [np.array(seq2seq_results)[indexes_to_keep]]

In [ ]:
def vec_2_str(vec, classes):
    ret = []
    for item in vec:
        temp = []
        for x in range(len(classes)):
            if item[x]==1:
                temp.append(classes[x])
        ret.append(' '.join(temp))
    return ret

In [ ]:
# s = vec_2_str(rf_tests[0][1], classes)
# with open('ttt_temp.txt','w') as f:
#     for x in s:
#         f.write(x+"\r\n")

In [ ]:
# sams= []
# with open('../files/seq2seq/home_os_final_test_services_to_keep_0_label') as f:
#     temp = f.readlines()
#     temp = [x.strip() for x in temp]
#     temp = [' '.join( sorted(x.split(' ')) ) for x in temp]
#     sams = temp

    
# for i in range(len(s)):
#     if s[i] != sams[i]:
#         print(i+1,"|", s[i],"|",sams[i])

In [ ]:
len(rf_tests[0][1]), len(seq2seq_results), len(seq2seq_results[0])

In [ ]:
for i in range(len(seq2seq_results)) :
    print( "==================HOME Case : %s =============" % test_names[ i] )
    pred = np.array(seq2seq_results[i])
    print_info( rf_tests[i][1], pred, classes, confidance=0.09)

## Show comparative resutls for xgb and RF

In [ ]:
def create_mask(for_classes, classes):
    ret = np.zeros(len(classes))
    
    for x in for_classes:
        ret[classes.index(x)] = 1

    return ret

xgb_classes = [
    'button',
    'level',
 'lock',
 
    'switch',
 
    
 ]
rf_classes =[
   
    
]
lstm_classes = [
    'contact',
    'ping',
 'status',
    
]
seq2seq_classes=[
    'colorTemperature',
    'motion',
    'temperature',
    'unknown'
]

rf_mask =create_mask(rf_classes, classes)
xgb_mask = create_mask(xgb_classes, classes)
lstm_mask = create_mask(lstm_classes, classes)
seq2seq_mask = create_mask(seq2seq_classes, classes)

ensambled_Resutls = []
for i in range(len(rf_test_known)) :
    print( "==================HOME Case : %s =============" % test_names[ i] )
    the_pred = rf_preds[i] * rf_mask + xgb_preds[i] * xgb_mask + lstm_preds[i]* lstm_mask + seq2seq_results[i] * seq2seq_mask
#     rf_pred = add_unknowns_back(rf_pred,rf_tests[i][1], rf_test_known[i][3] , classes)
#     rf_preds.append(rf_pred)
    ensambled_Resutls.append( print_info( rf_tests[i][1], the_pred, classes, confidance=0.09))

In [ ]:
# values, desc = print_info( rf_test_known[i][1], rf_pred, classes, confidance=0.8)

In [ ]:
def fix_nan(inp):
    if np.isnan(inp):
        return 0
    else:
        return inp
desc = xg_boost_results[0][1]
index = 0 
for index in range(len(test_names)):
    plt.figure(figsize=(14,4))
    plt.plot( [x[0] for x in xg_boost_results[index][0]] , 
         [fix_nan(x[desc.index("F Score")]) for x in xg_boost_results[index][0]],
        label="XGBoost",
         color="red" ,
             alpha=0.5
        )
#     plt.plot( [x[0] for x in rf_results[index][0]] , 
#          [fix_nan(x[desc.index("F Score")]) for x in rf_results[index][0]],
#         label="RF",
#          color="blue" ,
#              alpha=0.5
#         )
    plt.plot( [x[0] for x in rf_results[index][0]] , 
     [fix_nan(x[desc.index("F Score")]) for x in ensambled_Resutls[index][0]],
    label="Ensamble ",
     color="green" ,
         alpha=0.5
    )
    
    plt.plot( [x[0] for x in xg_boost_results[index][0]] , 
         [fix_nan(x[desc.index("Precision")]) for x in xg_boost_results[index][0]],
        label="XGBoost - Precision",
         color="red" ,
             marker="s",
             alpha=0.5
        )
#     plt.plot( [x[0] for x in rf_results[index][0]] , 
#          [fix_nan(x[desc.index("Precision")]) for x in rf_results[index][0]],
#         label="RF - Precision",
#          color="blue" ,
#              marker="s",
#              alpha=0.5
#         )
    plt.plot( [x[0] for x in rf_results[index][0]] , 
     [fix_nan(x[desc.index("Precision")]) for x in ensambled_Resutls[index][0]],
    label="Ensambled - Precision",
     color="green" ,
         marker="s",
         alpha=0.5
    )
    plt.plot( [x[0] for x in xg_boost_results[index][0]] , 
         [fix_nan(x[desc.index("Recall")]) for x in xg_boost_results[index][0]],
        label="XGBoost - Recall",
         color="red" ,
             marker="+",
             alpha=0.5
        )
#     plt.plot( [x[0] for x in rf_results[index][0]] , 
#          [fix_nan(x[desc.index("Recall")]) for x in rf_results[index][0]],
#         label="RF - Recall",
#          color="blue" ,
#              marker="+",
#              alpha=0.5
#         )

    plt.plot( [x[0] for x in rf_results[index][0]] , 
     [fix_nan(x[desc.index("Recall")]) for x in ensambled_Resutls[index][0]],
    label="Ensambled - Recall",
     color="green" ,
         marker="+",
         alpha=0.5
    )

    # plt.plot( )
    plt.legend()
    plt.xticks(rotation=90, ha='right')
    plt.title(test_names[ index] )
    plt.grid()

In [ ]:
from IPython.display import Audio, display

def allDone():
    display(Audio(url='https://sound.peal.io/ps/audios/000/000/537/original/woo_vu_luvub_dub_dub.wav', autoplay=True))
allDone()